In [13]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [21]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## This project analyzes and extract the main important facts regarding the countries around the world
Here are the descriptions for some of the columns:

- **name** — the name of the country.
- **area** — the total land and sea area of the country.
- **population** — the country's population.
- **population_growth** — the country's population growth as a percentage.
- **birth_rate** — the country's birth rate, or the number of births a year per 1,000 people.
- **death_rate** — the country's death rate, or the number of death a year per 1,000 people.
- **area** — the country's total area (both land and water).
- **area_land** — the country's land area in square kilometers.
- **area_water** — the country's water area in square kilometers.

In [33]:
%%sql
SELECT MIN(population) AS min_pop, MAX(population) AS max_pop, MIN(population_growth) AS min_pop_grow, MAX(population_growth) AS max_pop_grow
    FROM facts

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_grow,max_pop_grow
0,7256490011,0.0,4.02


There are some strange statistics regarding both the population and population growth as min_pop=0 and max_pop is over 7.2 billion which is almost 91% of current world population. Therefore, let's discover what countries are these by using more (sub)queries without using explicit number so that in future when the number changes we could detect dynamically

In [42]:
%%sql
SELECT *
  FROM facts
  WHERE population == (SELECT MIN(population)  FROM facts)


 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


Well, apparently, there are no inhabitants currently living in Antarctica as per our knowledge therefore the query above should be correct. Now let's observe the maximum population

In [43]:
%%sql
SELECT *
  FROM facts
  WHERE population == (SELECT MAX(population)  FROM facts)


 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Seemingly we will have to delete the row with **World** which shows the population recorded at that time. <br/> Basically 91% of current world population is a kind of ratio of world population at that time by today. <br/> Now let's exclude the outliers and recalculate

In [62]:
%%sql
SELECT MIN(population) AS min_pop, MAX(population) AS max_pop, MIN(population_growth) AS min_pop_grow, MAX(population_growth) AS max_pop_grow
    FROM facts
    WHERE name != 'Antarctica' AND name != 'World'

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_grow,max_pop_grow
48,1367485388,0.0,4.02


In [63]:
%%sql
SELECT AVG(population), AVG(area)
  FROM facts



 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


In [97]:
%%sql
SELECT *
    FROM facts
    WHERE population > (SELECT AVG(population)  
                          FROM facts 
                         WHERE name !='World') 
                           AND area<(SELECT AVG(area)  
                                       FROM facts  
                                      WHERE name !='World')
    ORDER BY birth_rate DESC

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


In [76]:
%%sql
SELECT name, MAX(population)
  FROM facts
  WHERE name !='World'

 * sqlite:///factbook.db
Done.


name,MAX(population)
China,1367485388


In [77]:
%%sql
SELECT name, MAX(population_growth)
  FROM facts
  WHERE name !='World'

 * sqlite:///factbook.db
Done.


name,MAX(population_growth)
South Sudan,4.02


In [85]:
%%sql
SELECT name, CAST(area_water AS FLOAT)/ area_land AS water_to_land
  FROM facts
  WHERE water_to_land>1

 * sqlite:///factbook.db
Done.


name,water_to_land
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832


In [86]:
%%sql
SELECT name, MAX(birth_rate)
  FROM facts
  WHERE name !='World'

 * sqlite:///factbook.db
Done.


name,MAX(birth_rate)
Niger,45.45


In [100]:
%%sql
SELECT name, CAST(death_rate AS FLOAT)/birth_rate AS death_birth_ratio
  FROM facts
  WHERE name !='World' and death_birth_ratio>1

 * sqlite:///factbook.db
Done.


name,death_birth_ratio
Austria,1.0010626992561105
Belarus,1.2485981308411216
Bosnia and Herzegovina,1.0992108229988726
Bulgaria,1.6188340807174888
Croatia,1.288888888888889
Czech Republic,1.0737279335410175
Estonia,1.1798287345385348
Germany,1.3482880755608027
Greece,1.2806004618937643
Hungary,1.3897379912663756


In [117]:
%%sql
SELECT COUNT(*) as num_countries
    FROM facts
    WHERE death_rate > birth_rate
    AND death_rate IS NOT NULL
    AND birth_rate IS NOT NULL;

 * sqlite:///factbook.db
Done.


num_countries
24


In [91]:
%%sql
SELECT name, CAST(population as FLOAT)/area as pop_area_ratio
  FROM facts
  WHERE name !='World'
  ORDER BY pop_area_ratio DESC
  LIMIT 10 

 * sqlite:///factbook.db
Done.


name,pop_area_ratio
Macau,21168.964285714286
Monaco,15267.5
Singapore,8141.279770444763
Hong Kong,6445.041516245487
Gaza Strip,5191.819444444444
Gibraltar,4876.333333333333
Bahrain,1771.8592105263158
Maldives,1319.6409395973155
Malta,1310.01582278481
Bermuda,1299.9259259259259


## CONCLUSION


In [145]:
%%sql
SELECT COUNT(*)
    FROM facts
    WHERE population/area>
      (SELECT AVG(CAST(population AS FLOAT)/area)
                   FROM facts)


 * sqlite:///factbook.db
Done.


COUNT(*)
25


- Which country has the most people? Which country has the highest growth rate?
<br>**Answer:** **China** has the most people of *1,367,485,388* and the *highest growth* rate was noticed in  **South Sudan**
<br/><br/>
- Which countries have the highest ratios of water to land? Which countries have more water than land? 
<br>**Answer:** These countries has the highest and only highest values of ratios of water to land areas: **British Indian Ocean Territory** and **Virgin islands**
<br/><br/>
- Which countries will add the most people to their populations next year?
<br>**Answer:** In order to check that, it is just enough to find the maximum birth rate, so the country is **Nigeria**
<br/><br/>
- Which countries have a higher death rate than birth rate?
<br>**Answer:** There are total **24** countries with death_rate>birth_rate
<br/><br/>
- Which countries have the highest population density and how many of them are higher than the average population density?
<br>**Answer:** There are total **25** countries with higher than the average population density listing from Macau to Bermuda